# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

T. Suhasaria  ->  T. Suhasaria  |  ['T. Suhasaria']
D. Fasano  ->  D. Fasano  |  ['D. Fasano']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
F. Zagaria  ->  F. Zagaria  |  ['F. Zagaria']
K. Doi  ->  K. Doi  |  ['K. Doi']
F. Walter  ->  F. Walter  |  ['F. Walter']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
N. Wang  ->  N. Wang  |  ['N. Wang']
Arxiv has 67 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.11596


extracting tarball to tmp_2506.11596... done.


T. Suhasaria  ->  T. Suhasaria  |  ['T. Suhasaria']


Found 61 bibliographic references in tmp_2506.11596/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2506.11688
extracting tarball to tmp_2506.11688... done.
Retrieving document from  https://arxiv.org/e-print/2506.11709
extracting tarball to tmp_2506.11709...

...

F. Walter  ->  F. Walter  |  ['F. Walter']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/LGLBS_overview_plot_v5
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure figures/LGLBS_overview_plot_v5 as tmp_2506.11792/./figures/LGLBS_overview_plot_v5.jpg
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 198 bibliographic references in tmp_2506.11792/ms.bbl.
Issues with the citations
syntax error in line 508: '=' expected
Retrieving document from  https://arxiv.org/e-print/2506.11915


extracting tarball to tmp_2506.11915...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.11596-b31b1b.svg)](https://arxiv.org/abs/2506.11596) | **Hydroxylated Mg-rich amorphous silicates as catalysts for molecular hydrogen formation in the interstellar medium**  |
|| V. Mennella, <mark>T. Suhasaria</mark>, B. Kerkeni, G. Ouerfelli |
|*Appeared on*| *2025-06-16*|
|*Comments*| *10 pages, 6 figures*|
|**Abstract**|            We present results from an experimental study on the interaction of atomic deuterium with Mg-rich amorphous enstatite and forsterite type silicates. Infrared spectroscopy was used to examine the process. During D atom exposure, deuteroxyl group formation was observed. The cross section for OD group formation, estimated from the OD stretching band intensity with D atom fluence, is 4.2.10^{-18} cm^2 for both silicates. HD (D2) molecules form via D atom abstraction of chemisorbed H(D) atoms from OH (OD) groups, with a cross section of 7.0.10^{-18} cm^2. Quantum chemical calculations on enstatite and forsterite amorphous nano-clusters were used to analyze chemisorption and abstraction energies of H atoms. The formation of OH groups on forsterite is barrierless, while enstatite has a small energy barrier. H2 abstraction from hydroxyl groups is barrierless in both silicates. The results support our interpretation of IR spectral changes during D atom exposure as addition and exchange reactions, with catalytic molecular deuterium formation. These findings, obtained at 300 K, are relevant to astrophysical environments like photodissociation regions and interstellar clouds at lower temperatures. Amorphous silicates, like carbon grains, undergo hydrogenation in the diffuse interstellar medium due to interactions with atomic hydrogen and UV photons. The detection of these components on comet 67P/Churyumov-Gerasimenko connects primitive solar system objects to interstellar dust, offering new insights into their evolution.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.11709-b31b1b.svg)](https://arxiv.org/abs/2506.11709) | **Inner disc and circumplanetary material in the PDS 70 system**  |
|| <mark>D. Fasano</mark>, et al. -- incl., <mark>M. Benisty</mark>, <mark>F. Zagaria</mark>, <mark>K. Doi</mark> |
|*Appeared on*| *2025-06-16*|
|*Comments*| *12 pages, 5 figures. Accepted for publication in A&A June 04, 2025*|
|**Abstract**|            The two giant protoplanets directly imaged in the dust-depleted cavity of PDS 70 offer a unique opportunity to study ongoing planet formation. Both planets have been detected in infrared thermal emission and in H$\alpha$, indicating active accretion. We calibrate and analyse archival ALMA Band 6 and 7 observations of PDS 70 from 2019, 2021, and 2023 to search for circumplanetary material and assess its motion. Using 2D visibility modelling of the high-resolution (~0.11"x0.08" in Band 6; ~0.05"x0.05" in Band 7) dust continuum from the outer disc, we subtract the model and image the cavity at multiple epochs. We re-detect compact dust emission around PDS 70 c in all datasets with >$3.8\sigma$ significance, and tentatively detect emission near PDS 70 b at ~$3\sigma$ in Band 6, with peak fluxes of $59\pm17\mu$Jy/beam and $46\pm14\mu$Jy/beam. The relative astrometry of the compact emission around PDS 70 c is consistent with the expected position of the planet between 2019-2023. We find a peak flux difference up to $64\pm34\mu$Jy/beam at 1$\sigma$, but Bayesian analysis indicates no significant variability. We detect no flux variability in the inner disc. The inferred dust mass near PDS 70 c and in the inner disc ranges from $0.008$-$0.063 M_\oplus$ and $0.04$-$0.31 M_\oplus$, respectively, consistent with prior estimates. Finally, we measure Band 6-7 spectral indices of $2.5\pm1.2$ (PDS 70 c) and $3.2\pm0.5$ (inner disc), suggesting that the inner disc emission is dominated by optically thin dust.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.11792-b31b1b.svg)](https://arxiv.org/abs/2506.11792) | **The Karl G. Jansky Very Large Array Local Group L-band Survey (LGLBS)**  |
|| E. W. Koch, et al. -- incl., <mark>F. Walter</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-06-16*|
|*Comments*| *ApJS in press. LGLBS HI v1.0 data release is available here: this https URL (with permanent DOI to follow)*|
|**Abstract**|            We present the Local Group L-Band Survey (LGLBS), a Karl G. Jansky Very Large Array (VLA) survey producing the highest quality 21-cm and 1-2 GHz radio continuum images to date for the six VLA-accessible, star-forming, Local Group galaxies. Leveraging the VLA's spectral multiplexing power, we simultaneously survey the 21-cm line at high 0.4 km/s velocity resolution, the 1-2 GHz polarized continuum, and four OH lines. For the massive spiral M31, the dwarf spiral M33, and the dwarf irregular galaxies NGC6822, IC10, IC1613, and the Wolf-Lundmark-Melotte Galaxy (WLM), we use all four VLA configurations and the Green Bank Telescope to reach angular resolutions of $< 5''$ ($10{-}20$~pc) for the 21-cm line with $<10^{20}$~cm$^{-2}$ column density sensitivity, and even sharper views ($< 2''$; $5{-}10$~pc) of the continuum. Targeting these nearby galaxies ($D\lesssim1$ Mpc) reveals a sharp, resolved view of the atomic gas, including 21-cm absorption, and continuum emission from supernova remnants and HII regions. These datasets can be used to test theories of the abundance and formation of cold clouds, the driving and dissipation of interstellar turbulence, and the impact of feedback from massive stars and supernovae. Here, we describe the survey design and execution, scientific motivation, data processing, and quality assurance. We provide a first look at and publicly release the wide-field 21-cm HI data products for M31, M33, and four dwarf irregular targets in the survey, which represent some of the highest physical resolution 21-cm observations of any external galaxies beyond the LMC and SMC.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.11688-b31b1b.svg)](https://arxiv.org/abs/2506.11688) | **The Energy Spectrum of Ultra-High Energy Cosmic Rays across Declinations $-90^\circ$ to $+44.8^\circ$ as measured at the Pierre Auger Observatory**  |
|| P. A. Collaboration, et al. |
|*Appeared on*| *2025-06-16*|
|*Comments*| *12 pages, 4 figures*|
|**Abstract**|            The energy spectrum of cosmic rays above 2.5 EeV has been measured across the declination range $-90^\circ \leq\delta\leq +44.8^\circ$ using data from $\sim 310{,}000$ events accrued at the Pierre Auger Observatory from an exposure of $(104{,}900\pm 3{,}100)$ km$^2\,$sr$\,$yr. No significant variations of energy spectra with declination are observed, after allowing or not for non-uniformities across the sky arising from the well-established dipolar anisotropies in the arrival directions of ultra-high energy cosmic rays. Additionally, the instep feature in the spectrum at $\simeq$ 10 EeV reported previously is now established at a significance above $5\,\sigma$. The quasi-uniformity of the energy spectrum across declinations disfavors an origin for the instep from a few distinctive sources.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.11915-b31b1b.svg)](https://arxiv.org/abs/2506.11915) | **A Hyperactive FRB Pinpointed in an SMC-Like Satellite Host Galaxy**  |
|| M. Bhardwaj, et al. -- incl., <mark>N. Wang</mark> |
|*Appeared on*| *2025-06-16*|
|*Comments*| *Submitted to ApJL. Comments welcome*|
|**Abstract**|            Precise localizations of fast radio bursts (FRBs) are essential for uncovering their host galaxies and immediate environments. We present the milliarcsecond-precision European VLBI Network (EVN) localization of FRB 20240114A, a hyper-active repeating FRB, achieving <90x30 mas (1-sigma) accuracy. This precision places the burst 0.5 kpc from the nucleus of its low-metallicity star-forming dwarf host at a spectroscopic redshift of z = 0.1300. Our Gran Telescopio CANARIAS (GTC) spectroscopic follow-up reveals that the dwarf FRB host is gravitationally bound to a more massive, star-forming spiral galaxy. This establishes the first known instance of an FRB residing in a satellite galaxy within a larger galactic system. This configuration, analogous to the Small Magellanic Cloud orbiting the Milky Way (but at a lower overall mass scale), expands the known diversity of FRB host environments and offers important insights for interpreting seemingly "hostless" or highly offset FRBs. Furthermore, our detailed dispersion measure (DM) budget analysis indicates that the dominant contribution to FRB 20240114A's DM likely originates from the foreground halo of the central galaxy. This finding addresses the anomalously high DM observed for this FRB and underscores the significant role of intervening foreground structures in shaping observed FRB DMs, which is important for accurate FRB-based cosmological measurements. Our results highlight the importance of deep, high-resolution optical/infrared observations (e.g., with Hubble or James Webb Space Telescopes) to fully leverage our precise radio localization and probe the immediate astrophysical birthplaces of FRB progenitors within these complex galactic systems.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.11596/./fig2.png', 'tmp_2506.11596/./fig6.png', 'tmp_2506.11596/./fig1.png']
copying  tmp_2506.11596/./fig2.png to _build/html/
copying  tmp_2506.11596/./fig6.png to _build/html/
copying  tmp_2506.11596/./fig1.png to _build/html/
exported in  _build/html/2506.11596.md
    + _build/html/tmp_2506.11596/./fig2.png
    + _build/html/tmp_2506.11596/./fig6.png
    + _build/html/tmp_2506.11596/./fig1.png
found figures ['tmp_2506.11709/./Images/Summary_plots.png', 'tmp_2506.11709/./Images/Residual_cavity_all_B7.png', 'tmp_2506.11709/./Images/Residual_cavity_all_B6.png', 'tmp_2506.11709/./Images/Residual_cavity_clumps_B7.png']
copying  tmp_2506.11709/./Images/Summary_plots.png to _build/html/
copying  tmp_2506.11709/./Images/Residual_cavity_all_B7.png to _build/html/
copying  tmp_2506.11709/./Images/Residual_cavity_all_B6.png to _build/html/
copying  tmp_2506.11709/./Images/Residual_cavity_clumps_B7.png to _build/html/
exported in  _build/html/2506.11709.md
    + _build

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Hydroxylated Mg-rich amorphous silicates as catalysts for molecular hydrogen formation in the interstellar medium

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.11596-b31b1b.svg)](https://arxiv.org/abs/2506.11596)<mark>Appeared on: 2025-06-16</mark> -  _10 pages, 6 figures_

</div>
<div id="authors">

V. Mennella, <mark>T. Suhasaria</mark>, B. Kerkeni, G. Ouerfelli

</div>
<div id="abstract">

**Abstract:** We present results from an experimental study on the interaction of atomic deuterium with Mg-rich amorphous enstatite and forsterite type silicates. Infrared spectroscopy was used to examine the process. During D atom exposure, deuteroxyl group formation was observed. The cross section for OD group formation, estimated from the OD stretching band intensity with D atom fluence, is $\sigma _{f,OD}$ = 4.2 $\times10^{-18}$ cm $^{2}$ for both silicates. HD ($D_2$ ) molecules form via D atom abstraction of chemisorbed H (D) atoms from OH (OD) groups, with a cross section of $\sigma _{f,D_2}$ = 7.0 $\times10^{-18}$ cm $^{2}$ . Quantum chemical calculations on enstatite and forsterite amorphous nano-clusters were used to analyze chemisorption and abstraction energies of H atoms. The formation of OH groups on forsterite is barrierless, while enstatite has a small energy barrier. $H_2$ abstraction from hydroxyl groups is barrierless in both silicates. The results support our interpretation of IR spectral changes during D atom exposure as addition and exchange reactions, with catalytic molecular deuterium formation. These findings, obtained at 300 K, are relevant to astrophysical environments like photodissociation regions and interstellar clouds at lower temperatures. Amorphous silicates, like carbon grains, undergo hydrogenation in the diffuse interstellar medium due to interactions with atomic hydrogen and UV photons. The detection of these components on comet 67P/Churyumov-Gerasimenko connects primitive solar system objects to interstellar dust, offering new insights into their evolution.

</div>

<div id="div_fig1">

<img src="tmp_2506.11596/./fig2.png" alt="Fig2" width="100%"/>

**Figure 2. -** A) Evolution of the O-H and O-D stretching bands during D atom irradiation of the SIL1 and SIL2 silicates. The spectra  of the exposed silicates are shown after subtraction
of the spectrum of the samples as produced. The SIL1 spectra are offset in ordinate for the sake of clarity.
B) The O-D stretching band of SIL1 after D exposure compared with the O-H mode of SIL1 as shown in the inset to Figure \ref{fig1}. The later has been red shifted, reducing the frequency of a factor 1.36, to take into account the H/D isotope substitution.
  (*fig2*)

</div>
<div id="div_fig2">

<img src="tmp_2506.11596/./fig6.png" alt="Fig6" width="100%"/>

**Figure 6. -** The ER potential energy surface along the minimum energy path for (a) H atom approaching an hydroxyl group from infinity and (b) $H_2$ molecule abstraction from the enstatite nanocluster. (*fig6*)

</div>
<div id="div_fig3">

<img src="tmp_2506.11596/./fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Continuum subtracted mid-IR spectra of SIL1 and SIL2. The details of the O-H stretching region are shown in the inset. The spectra are offset in ordinate for the sake of clarity.  (*fig1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.11596"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Daniele}[1]{{\color{MidnightBlue}\textbf{DF}: #1}}$
$\newcommand{\mb}[1]{{\color{magenta}\textbf{mb}: #1}}$
$\newcommand{\Tomo}[1]{{\color{ForestGreen}\textbf{TY}: #1}}$
$\newcommand{\fsco}[1]{{\color{Orange}\textbf{fsco}: #1}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# Inner Disc and Circumplanetary Material in the PDS 70 System: Insights from Multi-Epoch, Multi-Frequency ALMA Observations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.11709-b31b1b.svg)](https://arxiv.org/abs/2506.11709)<mark>Appeared on: 2025-06-16</mark> -  _12 pages, 5 figures. Accepted for publication in A&A June 04, 2025_

</div>
<div id="authors">

<mark>D. Fasano</mark>, et al. -- incl., <mark>M. Benisty</mark>, <mark>F. Zagaria</mark>, <mark>K. Doi</mark>

</div>
<div id="abstract">

**Abstract:** The two giant protoplanets directly detected in the dust-depleted cavity of PDS 70 offer a unique opportunity to study the processes of ongoing planet formation. The planets have been detected both in IR thermal light and in the H $\alpha$ line, indicating that they are actively accreting material from their surroundings. We calibrate and analyse archival Atacama Large Millimetre/subMillimetre Array (ALMA) Band 6 and 7 observations of PDS 70 to detect circumplanetary material in independent datasets taken at different epochs in 2019, 2021 and 2023 and assess its possible motion. We perform 2D visibility modelling of the high resolution ALMA Band 6 ( $\sim0.11\arcsec\times0.08\arcsec$ ) and Band 7 ( $\sim0.05\arcsec\times0.05\arcsec$ )  dust continuum emission of the outer disc. After subtracting the model from the data, we image the dust continuum emission in the cavity of PDS 70 at multiple epochs. We re-detect the compact dust emission around PDS 70 c in all our datasets in Band 6 and 7, with more than $3.8\sigma$ significance, and tentatively detect compact emission around PDS 70 b at $\sim3\sigma$ in the Band 6 datasets, with a peak emission of $59\pm17 \rm\mu Jy/beam$ and $46\pm14 \rm\mu Jy/beam$ . We find the astrometric relative position of the compact emission around PDS 70 c to be consistent with the expected position of the planet in the 2019-2023 time range. We measure a peak flux difference of up to $64\pm34\rm \mu Jy/beam$ at $1\sigma$ confidence level for the continuum emission coming from the region around PDS 70 c and perform a Bayesian test on our measurements, finding that they are not consistent with significant variable emission. We find no evidence of flux variability in the inner disc. We measure the dust mass of the material co-located with PDS 70 c and the inner disc to be in the range $0.008-0.063 M_\oplus$ and $0.04-0.31 M_\oplus$ , respectively, consistent with previous measurements. Additionally, we obtain Band 6-7 spectral indices of $2.5\pm1.2$ and $3.2\pm0.5$ for the dust emission around PDS 70 c and in the inner disc, respectively, suggesting the inner disc emission is dominated by optically thin dust.

</div>

<div id="div_fig1">

<img src="tmp_2506.11709/./Images/Summary_plots.png" alt="Fig2" width="100%"/>

**Figure 2. -** Left to right: Continuum images with insets showing a zoom in the cavity; CLEANed \texttt{galario} model images; CLEANed residual images; zoom on the North (1) and South (2) sides of the outer ring residuals. Contours start at $3\sigma$ and increase by $1\sigma$. The green ellipses approximate the $7\sigma$ contours of the continuum data. The datasets are ordered from Band 7 to Band 6 and by epoch from top to bottom. (*fig:Summary plots*)

</div>
<div id="div_fig2">

<img src="tmp_2506.11709/./Images/Residual_cavity_all_B7.png" alt="Fig3.1" width="50%"/><img src="tmp_2506.11709/./Images/Residual_cavity_all_B6.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** Zoom of the residuals in the cavity from Fig.\ref{fig:Summary plots}. a) Band 7 2019 (left panel), 2021 (middle panel) and 2023 (right panel) datasets. b) Band 6 2021 datasets at $\rm220 GHz$(left panel) and $\rm260 GHz$(right panel). The solid contours start from the $3\sigma$ emission and increase by $1\sigma$, while the  dashed contours correspond to the $3\sigma$ emission from the B7 2023 dataset and are overlaid to the other datasets for comparison. The white plus sign marks the centre of the disc, corrected with the \texttt{galario} offset. The green circles and purple diamonds represent the expected position in July 2019 (dark), July 2021 (medium) and May 2023 (light) obtained from the orbital fit of PDS 70 b and c  ([Wang, Vigan and Lacour 2021]()) , respectively. The error bars are smaller than the markers.
               (*fig:Residual cavity all*)

</div>
<div id="div_fig3">

<img src="tmp_2506.11709/./Images/Residual_cavity_clumps_B7.png" alt="Fig4" width="100%"/>

**Figure 4. -** Same as left an middle panels of Fig. \ref{fig:Residual cavity all B7}, with additional labels to identify Clump 1 and Clump 2. The dashed white and black circles represent the location where we re-detect Clump 1 and Clump 2, respectively, in the B7 2019 dataset (left panel), and their expected position assuming Keplerian rotation in the B7 2021 dataset (right panel). (*fig:Inner disc clumps*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.11709"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\msolpcsq}{M_\odot pc^{-2}\xspace}$
$\newcommand{\msolyr}{M_\odot yr^{-1}\xspace}$
$\newcommand{\msol}{M_\odot\xspace}$
$\newcommand{\twelveco}{^{12}CO\xspace}$
$\newcommand{\twelvecoto}{^{12}CO\mbox{(2-1)}\xspace}$
$\newcommand{\twelvecotht}{^{12}CO\mbox{(3-2)}\xspace}$
$\newcommand{\thirtco}{^{13}CO\xspace}$
$\newcommand{\thirtcoto}{^{13}CO\mbox{(2-1)}\xspace}$
$\newcommand{\thirtcotht}{^{13}CO\mbox{(3-2)}\xspace}$
$\newcommand{\ceighto}{C^{18}O\xspace}$
$\newcommand{\hcop}{HCO^{+}\xspace}$
$\newcommand{\hi}{\ion{H}{1}\xspace}$
$\newcommand{◦ee}{\ensuremath{\circ}}$
$\newcommand{\hii}{{\sc H\textsc{ ii}}\xspace}$
$\newcommand{\kms}{km s^{-1}\xspace}$
$\newcommand{\htwo}{H_{2}\xspace}$
$\newcommand{\halpha}{H\alpha\xspace}$
$\newcommand{\Msun}{M_{\odot}\xspace}$
$\newcommand{\Ox}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\UGent}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\STScI}{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\IWR}{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, 69120 Heidelberg, Germany}$
$\newcommand{\Radcliffe}{Elizabeth S. and Richard M. Cashin Fellow at the Radcliffe Institute for Advanced Studies at Harvard University, 10 Garden Street, Cambridge, MA 02138, U.S.A.}$
$\newcommand{\MPIA}{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117, Heidelberg, Germany}$
$\newcommand{\AURA}{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$\newcommand{\UCSD}{Department of Astronomy \& Astrophysics, University of California, San Diego, 9500 Gilman Dr., La Jolla, CA 92093, USA}$
$\newcommand{\Whitman}{Whitman College, 345 Boyer Avenue, Walla Walla, WA 99362, USA}$
$\newcommand{\JHU}{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218, USA}$
$\newcommand{\OSU}{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, OH 43210, USA}$
$\newcommand{\OSUPhys}{Department of Physics, The Ohio State University, Columbus, Ohio 43210, USA}$
$\newcommand{ÇAPP}{Center for Cosmology and Astroparticle Physics (CCAPP), 191 West Woodruff Avenue, Columbus, OH 43210, USA}$
$\newcommand{\ARI}{Astronomisches Rechen-Institut, Zentrum für Astronomie der Universität Heidelberg, Mönchhofstr. 12-14, D-69120 Heidelberg, Germany}$
$\newcommand{\ANU}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\UConn}{Department of Physics, University of Connecticut, 196A Auditorium Road, Storrs, CT 06269, USA}$
$\newcommand{\UHawaii}{Institute for Astronomy, University of Hawaii, 2680 Woodlawn Drive, Honolulu, HI 96822, USA}$
$\newcommand{\UniCA}{Université C\^{o}te d'Azur, Observatoire de la C\^{o}te d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}$
$\newcommand{\UAlberta}{Dept. of Physics, University of Alberta, 4-183 CCIS, Edmonton, Alberta, T6G 2E1, Canada}$
$\newcommand{\Arcetri}{INAF — Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Florence, Italy}$
$\newcommand{\UWyoming}{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}$
$\newcommand{\LJMU}{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\newcommand{\ITA}{Universität Heidelberg, Zentrum für Astronomie, Institut für Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}$
$\newcommand{\CfA}{Center for Astrophysics \mid Harvard \& Smithsonian, 60 Garden St., 02138 Cambridge, MA, USA}$
$\newcommand{\MPE}{Max-Planck-Institut für Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\newcommand{\UMD}{Department of Astronomy and Joint Space-Science Institute, University of Maryland, College Park, MD 20742, USA}$
$\newcommand{\UVA}{Department of Astronomy, University of Virginia, Charlottesville, VA, USA}$
$\newcommand{\NRAO}{National Radio Astronomy Observatory, Charlottesville, VA, USA}$
$\newcommand{\ASIAA}{Institute of Astronomy and Astrophysics, Academia Sinica, No. 1, Sec. 4, Roosevelt Road, Taipei 106216, Taiwan}$
$\newcommand{\kipac}{Kavli Institute for Particle Astrophysics \& Cosmology (KIPAC), Stanford University, CA 94305, USA}$
$\newcommand{\aifa}{Argelander-Institut für Astronomie, University of Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\TKU}{Department of Physics, Tamkang University, No.151, Yingzhuan Road, Tamsui District, New Taipei City 251301, Taiwan}$
$\newcommand{\CarnegieObs}{The Observatories of the Carnegie Institution for Science. 813 Santa Barbara Street, Pasadena, CA 91101, USA}$
$\newcommand{\Princeton}{Department of Astrophysical Sciences, Princeton University, Princeton, NJ 08544, USA}$
$\newcommand{\IAS}{Institute for Advanced Study, 1 Einstein Drive, Princeton, NJ 08540, USA}$
$\newcommand{\COOL}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\ESO}{European Southern Observatory (ESO), Karl-Schwarzschild-Stra{\ss}e 2, 85748 Garching, Germany}$
$\newcommand{\ULyon}{Univ Lyon, Univ Lyon 1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval, France}$
$\newcommand{\UoM}{Jodrell Bank Centre for Astrophysics, Department of Physics and Astronomy, University of Manchester, Oxford Road, Manchester M13 9PL, UK}$
$\newcommand{\farcs}{\hbox{.\!\!^{\prime\prime}}}$
$\newcommand{\lesssim}{\mathrel{\hbox{\rlap{\hbox{\lower4pt\hbox{\sim}}}\hbox{<}}}}$
$\newcommand{\gtrsim}{\mathrel{\hbox{\rlap{\hbox{\lower4pt\hbox{\sim}}}\hbox{>}}}}$</div>



<div id="title">

# The Karl G. Jansky Very Large Array Local Group L-band Survey (LGLBS)

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.11792-b31b1b.svg)](https://arxiv.org/abs/2506.11792)<mark>Appeared on: 2025-06-16</mark> -  _ApJS in press. LGLBS HI v1.0 data release is available here: this https URL (with permanent DOI to follow)_

</div>
<div id="authors">

E. W. Koch, et al. -- incl., <mark>F. Walter</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We present the Local Group L-Band Survey (LGLBS), a Karl G. Jansky Very Large Array (VLA) survey producing the highest quality 21-cm and $1{-}2$ GHz radio continuum images to date for the six VLA-accessible, star-forming, Local Group galaxies. Leveraging the VLA's spectral multiplexing power, we simultaneously survey the 21-cm line at high $0.4$ km s $^{-1}$ velocity resolution, the 1--2 GHz polarized continuum, and four OH lines. For the massive spiral M31, the dwarf spiral M33, and the dwarf irregular galaxies NGC6822, IC10, IC1613, and the Wolf-Lundmark-Melotte Galaxy (WLM), we use all four VLA configurations and the Green Bank Telescope to reach angular resolutions of $< 5"$ ( $10{-}20$ pc) for the 21-cm line with $<10^{20}$ cm $^{-2}$ column density sensitivity, and even sharper views ( $< 2"$ ; $5{-}10$ pc) of the continuum. Targeting these nearby galaxies ( $D\lesssim1$ Mpc) reveals a sharp, resolved view of the atomic gas, including 21-cm absorption, and continuum emission from supernova remnants and $\ion{H}{2}$ regions. These datasets can be used to test theories of the abundance and formation of cold clouds, the driving and dissipation of interstellar turbulence, and the impact of feedback from massive stars and supernovae. Here, we describe the survey design and execution, scientific motivation, data processing, and quality assurance. We provide a first look at and publicly release the wide-field 21-cm $\hi$ data products for M31, M33, and four dwarf irregular targets in the survey, which represent some of the highest physical resolution 21-cm observations of any external galaxies beyond the LMC and SMC.

</div>

<div id="div_fig1">

<img src="tmp_2506.11792/./figures/dwarfs_c+d_ew.png" alt="Fig14" width="100%"/>

**Figure 14. -** **$\hi$ line width maps of the four LGLBS dwarf galaxies, as measured with the effective width,** defined as $\sigma = I_{\rm HI} / \sqrt{2\pi}   T_{\rm peak}$. Each map is shown at a common physical resolution of 120 pc. Regions with recent massive star formation in NGC6822 and IC10 tend to show larger velocity dispersions, while IC1613 and WLM with lower recent star formation rates tend to have lower dispersions. (*fig:hi_dwarfs_ew*)

</div>
<div id="div_fig2">

<img src="tmp_2506.11792/./figures/flagging_stats_by_spw.png" alt="Fig2" width="100%"/>

**Figure 2. -** **Median percent of flagged data per SPW, where the errorbars reflect the 15$^{\rm th**$ to 85$^{\rm th}$ percentile range of 337 LGLBS tracks}. The continuous blue lines indicate flagging percentages for the continuum SPWs, and the orange diamonds are the line SPWs for $\hi$ and the 4 OH lines (the OH1665/1667 are recorded together in the same SPW). The current LGLBS processing does not include the Hydrogen RRLs included in the spectral setup. (*fig:flagging_by_freq*)

</div>
<div id="div_fig3">

<img src="tmp_2506.11792/./figures/m31_m33_c+d_mom1.png" alt="Fig13" width="100%"/>

**Figure 13. -** **$\hi$ centroid velocity maps of M31 and M33** at a common physical resolution of 120 pc. The colormaps are centered at the galaxy's systemic velocity indicated by the vertical black line in each colorbar and extend to $\pm V_{\rm max}$(the maximum rotation velocity). (*fig:hi_m31m33_mom1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.11792"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

524  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
